In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# import dataset
data = pd.read_csv("housing.csv")

# shuffle dataset
data = data.sample(n=len(data))

# calculate ln of certain variables
data['total_rooms']= np.log(data['total_rooms']+1)
data['total_bedrooms']= np.log(data['total_bedrooms']+1)
data['population']= np.log(data['population']+1)
data['households']= np.log(data['households']+1)

# scale data using standard scaler
from sklearn.preprocessing import StandardScaler
data = pd.DataFrame(data)
columns_to_exclude = ['ocean_proximity']
scaler = StandardScaler()
scaled_data = data.copy()
scaled_data.loc[:, data.columns.difference(columns_to_exclude)] = scaler.fit_transform(data.loc[:, data.columns.difference(columns_to_exclude)])

# one hot vector encoding
scaled_data=scaled_data.join(pd.get_dummies(scaled_data.ocean_proximity)).drop(['ocean_proximity'], axis=1)

# fill missing values with median
scaled_data['total_bedrooms'] = scaled_data['total_bedrooms'].fillna(scaled_data['total_bedrooms'].median())


C:\Users\nikos\AppData\Local\Temp\ipykernel_40860\3017778517.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
class Perceptron:
    # initialization function for variables
    def __init__(self, learning_rate, n_iterations, threshold):
        self.learning_rate = learning_rate
        self.n_iterations = n_iterations
        self.threshold = threshold
        self.weights = None
        self.bias = None
        
    # training function
    def train(self, X, y):
        # initialize weights and bias with 0
        self.weights = np.zeros(X.shape[1])
        self.bias = 0

        # update weights and bias
        for _ in range(self.n_iterations):
            for xi, target in zip(X, y):
                update = self.learning_rate * (target - self.predict(xi))
                self.weights += update * xi
                self.bias += update
                
    def predict(self, X):
        # predict the class -1 or 1 based on the threshold
        return np.where((np.dot(X, self.weights) + self.bias) > self.threshold, 1, -1)



# kfold cross validation(10 folds)
from sklearn.model_selection import KFold
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# mae and mse
from sklearn.metrics import mean_absolute_error, mean_squared_error
mae_train_list, mse_train_list = [], []
mae_test_list, mse_test_list = [], []

# X y split
X = scaled_data.drop('median_house_value', axis=1)
y = scaled_data['median_house_value']

count = 1

# loop through each fold
for train_index, test_index in kf.split(X):
    print("Fold", count, ":")
          
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    # convert bool values to float64
    X_train, X_test = X_train.astype('float64'), X_test.astype('float64')
    
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    # calculate and print threshold for this fold
    kFold_threshold = np.median(y_train)
    print("Threshold for this Fold:", kFold_threshold)
    # convert y values to -1 or 1 based on the threshold
    y_train_bin, y_test_bin = np.where(y_train > kFold_threshold, 1, -1), np.where(y_test > kFold_threshold, 1, -1)

    
    
    # initialize and train perceptron model
    perceptron = Perceptron(learning_rate=0.2, n_iterations=2, threshold = kFold_threshold)
    perceptron.train(X_train.values, y_train)


    
    # evaluate model on train data
    predictions_train = perceptron.predict(X_train.values)
    # calculate mean absolute error
    mae_train = mean_absolute_error(y_train_bin, predictions_train)
    print("Mean Absolute Error for TRAIN set:", mae_train)
    mae_train_list.append(mae_train)
    # calculate mean squared error
    mse_train = mean_squared_error(y_train_bin, predictions_train)
    print("Mean Sqared Error for TRAIN set:", mse_train)
    mse_train_list.append(mse_train)

    # evaluate model on test data
    predictions_test = perceptron.predict(X_test.values)
    # calculate mean absolute error
    mae_test = mean_absolute_error(y_test_bin, predictions_test)
    print("Mean Absolute Error for TEST set:", mae_test)
    mae_test_list.append(mae_test)
    # calculate mean squared error
    mse_test = mean_squared_error(y_test_bin, predictions_test)
    print("Mean Sqared Error for TEST set:", mse_test, "\n")
    mse_test_list.append(mse_test)

    count += 1

# print average values
print("Average MAE for TRAIN set:", np.mean(mae_train_list))
print("Average MSE for TRAIN set:", np.mean(mse_train_list))
print("Average MAE for TEST set:", np.mean(mae_test_list))
print("Average MSE for TEST set:", np.mean(mse_test_list))


Fold 1 :
Threshold for this Fold: -0.23446707355068813
Mean Absolute Error for TRAIN set: 0.3481912144702842
Mean Sqared Error for TRAIN set: 0.6963824289405685
Mean Absolute Error for TEST set: 0.3682170542635659
Mean Sqared Error for TEST set: 0.7364341085271318 

Fold 2 :
Threshold for this Fold: -0.2327338634662169
Mean Absolute Error for TRAIN set: 0.3502368647717485
Mean Sqared Error for TRAIN set: 0.700473729543497
Mean Absolute Error for TEST set: 0.3488372093023256
Mean Sqared Error for TEST set: 0.6976744186046512 

Fold 3 :
Threshold for this Fold: -0.23186725842398132
Mean Absolute Error for TRAIN set: 0.3506675279931094
Mean Sqared Error for TRAIN set: 0.7013350559862188
Mean Absolute Error for TEST set: 0.3449612403100775
Mean Sqared Error for TEST set: 0.689922480620155 

Fold 4 :
Threshold for this Fold: -0.2327338634662169
Mean Absolute Error for TRAIN set: 0.3513135228251507
Mean Sqared Error for TRAIN set: 0.7026270456503014
Mean Absolute Error for TEST set: 0.348837